In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_datareader.data as pdr
from matplotlib import pyplot as plt

# Calculate real return

In [2]:
start="2019-12-1"
end="2023-1-1"
price = yf.download("GOOGL", start, end)["Adj Close"]
simple_return = price.pct_change()

data = (pd.DataFrame(simple_return)
        .resample("MS").first()
        .rename(columns={'Adj Close':"Simple_return"}))

cpi = (pdr.DataReader('CPIAUCSL', 'fred', start, end)
       .rename(columns={'CPIAUCSL':"CPI"}))

cpi = cpi.pct_change()

data = data.join(cpi, how="left")
data.dropna(inplace=True)
data["Real_return"] = (data["Simple_return"]+1)/(data["CPI"]+1) - 1
data

[*********************100%***********************]  1 of 1 completed


,Simple_return,CPI,Real_return
Date,,,
2020-01-01,0.021868,0.001628,0.020207
2020-02-01,0.034772,0.000815,0.033929
2020-03-01,0.035147,-0.004336,0.039654
2020-04-01,-0.051508,-0.007872,-0.043982
2020-05-01,-0.021816,-0.000875,-0.020960
2020-06-01,0.000942,0.004369,-0.003413
2020-07-01,0.016889,0.005028,0.011803
2020-08-01,-0.003488,0.004387,-0.007840
2020-09-01,0.015679,0.002382,0.013265


# Process OHLCV

In [3]:
from binance.client import Client

client=Client()

def get_historical_ohlc_data(symbol,past_days=None,interval=None):
    
    """Returns historcal klines from past for given symbol and interval
    past_days: how many days back one wants to download the data"""
    
    if not interval:
        interval='1m' # default interval 1 minute
    if not past_days:
        past_days=5  # default past 5 days.

    start_str=str((pd.to_datetime('today')-pd.Timedelta(str(past_days)+' days')).date())
    
    data=pd.DataFrame(client.get_historical_klines(symbol=symbol,start_str=start_str,interval=interval))
    data.columns=['open_time','open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades', 'Taker buy base asset volume', 'Taker buy quote asset volume','Ignore']
    data['open_date_time']=[dt.datetime.fromtimestamp(x/1000) for x in data.open_time]
    data['symbol']=symbol
    data=data[['symbol','open_date_time','open', 'high', 'low', 'close', 'volume', 'num_trades']]
    data[['open', 'high', 'low', 'close', 'volume', 'num_trades']] = (data[['open', 'high', 'low', 'close', 'volume', 'num_trades']]
                                                                      .apply(pd.to_numeric))

    return data

In [4]:
data_ohlcv = get_historical_ohlc_data(symbol="BTCUSDT")
data_ohlcv

,symbol,open_date_time,open,high,low,close,volume,num_trades
0,BTCUSDT,2023-06-28 01:00:00,30692.44,30709.74,30692.44,30698.96,26.39199,736
1,BTCUSDT,2023-06-28 01:01:00,30698.96,30698.97,30673.23,30673.24,11.01572,563
2,BTCUSDT,2023-06-28 01:02:00,30673.24,30680.45,30673.14,30680.45,20.89324,440
3,BTCUSDT,2023-06-28 01:03:00,30680.45,30680.45,30668.50,30668.57,13.03124,389
4,BTCUSDT,2023-06-28 01:04:00,30668.57,30674.47,30668.56,30674.44,18.97984,327
...,...,...,...,...,...,...,...,...
7886,BTCUSDT,2023-07-03 12:26:00,30676.74,30676.74,30668.38,30668.39,19.14374,439
7887,BTCUSDT,2023-07-03 12:27:00,30668.38,30668.39,30662.02,30662.03,16.03753,342
7888,BTCUSDT,2023-07-03 12:28:00,30662.02,30662.03,30651.30,30651.30,18.15895,434
7889,BTCUSDT,2023-07-03 12:29:00,30651.30,30652.17,30651.30,30652.16,32.68482,363


In [5]:
data = data_ohlcv.set_index("open_date_time")
typical_price = (data["high"]+data["low"]+data["close"])/3*data["volume"]
cum_typical_price = typical_price.to_frame().resample("30min").sum().cumsum()
cum_volume = data["volume"].to_frame().resample("30min").sum().cumsum()
vwap = cum_typical_price
vwap = vwap.join(cum_volume, how="left").rename(columns={0:"cum_price_vol","volume":"cum_vol"})
vwap["vwap"] = vwap["cum_price_vol"]/vwap["cum_vol"]
vwap

,cum_price_vol,cum_vol,vwap
open_date_time,,,
2023-06-28 01:00:00,3.475424e+07,1137.03817,30565.584101
2023-06-28 01:30:00,5.908126e+07,1933.84626,30551.169190
2023-06-28 02:00:00,7.706074e+07,2521.35434,30563.231102
2023-06-28 02:30:00,9.074500e+07,2969.74309,30556.516208
2023-06-28 03:00:00,1.010753e+08,3308.26937,30552.329094
...,...,...,...
2023-07-03 10:30:00,6.621442e+09,217372.71371,30461.237432
2023-07-03 11:00:00,6.640433e+09,217993.07442,30461.668557
2023-07-03 11:30:00,6.660056e+09,218633.30579,30462.219127
